In [1]:
#new chrome driver

from selenium import webdriver

def OpenChromeDriver():
    #開啟瀏覽器新分頁，下載google chrome driver
    ban_notify=webdriver.ChromeOptions()
    prefs={"profile.default_content_setting_values.notifications" : 2}
    ban_notify.add_experimental_option("prefs",prefs)
    gchrome=webdriver.Chrome(options=ban_notify)
    
    return gchrome

In [2]:
#login

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

import sys
import time
import getpass

def loginfb(driver):
    driver.get("https://www.facebook.com")
    
    username="***"
    pwd="***"
    
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="email"]')))
    tempentering=driver.find_element_by_id("email")
    tempentering.send_keys(username)

    tempentering=driver.find_element_by_id("pass")
    tempentering.send_keys(pwd)

    tempentering.send_keys(Keys.RETURN)
    time.sleep(5)

In [19]:
#抓取貼文留言

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from datetime import date
from bs4 import BeautifulSoup
import sys
import time
import pandas as pd


def getPostComment(url,driver,this_post_date,count):
    driver.get(url)
    time.sleep(2)

    old_btn=driver.find_element_by_css_selector("div.rq0escxv.l9j0dhe7.du4w35lb.nc684nl6.g0qnabr5")
    ActionChains(driver).move_to_element(old_btn).perform()
    old_btn.click()
    time.sleep(2)
    
    all_comment_btn=driver.find_element_by_css_selector("div > div > div:nth-child(2) > div > div > div.j34wkznp.qp9yad78.pmk7jnqg.kr520xx4 > div.iqfcb0g7.tojvnm2t.a6sixzi8.k5wvi7nf.q3lfd5jv.pk4s997a.bipmatt0.cebpdrjk.qowsmv63.owwhemhu.dp1hu0rb.dhp61c6y.l9j0dhe7.iyyx5f41.a8s20v7p > div > div > div.rq0escxv.jgsskzai.cwj9ozl2.nwpbqux9.io0zqebd.m5lcvass.fbipl8qg.nwvqtn77.ni8dbmo4.stjgntxs > div > div > div > div.j83agx80.cbu4d94t.buofh1pr.l9j0dhe7 > div > div:nth-child(3) > div.bp9cbjyn.j83agx80.btwxx1t3.buofh1pr.i1fnvgqd.hpfvmrgz > div > div:nth-child(1) > span")
    ActionChains(driver).move_to_element(all_comment_btn).perform()
    all_comment_btn.click()
    time.sleep(2)
    
    while True:
        try:
            try:
                driver.find_element_by_css_selector("div:nth-child(1) > div > div.rq0escxv.l9j0dhe7.du4w35lb > div > div > div.j83agx80.cbu4d94t.d6urw2fd.dp1hu0rb.l9j0dhe7.du4w35lb > div.l9j0dhe7.dp1hu0rb.cbu4d94t.j83agx80 > div.bp9cbjyn.j83agx80.cbu4d94t.d2edcug0 > div.d2edcug0.oh7imozk.tr9rh885.abvwweq7.ejjq64ki > div > div > div > div > div > div > div > div > div > div.rq0escxv.l9j0dhe7.du4w35lb.hybvsw6c.io0zqebd.m5lcvass.fbipl8qg.nwvqtn77.k4urcfbm.ni8dbmo4.stjgntxs.sbcfpzgs > div > div:nth-child(2) > div > div:nth-child(4) > div > div > div.cwj9ozl2.tvmbv18p > div:nth-child(6) > div.j83agx80.bp9cbjyn > span")
            except:
                break
            more_comment_btn=driver.find_element_by_css_selector("span.j83agx80.fv0vnmcu.hpfvmrgz")
            ActionChains(driver).move_to_element(more_comment_btn).perform()
            more_comment_btn.click()
            time.sleep(0.1)
        
        #頁面上無更多頁面按鈕
        except StaleElementReferenceException:
            try:
                driver.execute_script("window.scrollTo(0,200)")
                more_comment_btn=driver.find_element_by_css_selector("span.j83agx80.fv0vnmcu.hpfvmrgz")
                ActionChains(driver).move_to_element(more_comment_btn).perform()
                more_comment_btn.click()
                time.sleep(0.1)
                
            except StaleElementReferenceException:
                out_btn=driver.find_element_by_css_selector("div.bp9cbjyn.g5ia77u1.j83agx80.j0lfo8lj.taijpn5t.ocgrx2df.irj2b8pg.etr7akla")
                ActionChains(driver).move_to_element(out_btn).perform()
                
            except NoSuchElementException:
                driver.execute_script("window.scrollTo(0,200)")
                print("stop")
                break
            
        except NoSuchElementException:
            driver.execute_script("window.scrollTo(0,200)")
            print("stop")
            break
            
    #Get Comment       
    soup=BeautifulSoup(driver.page_source,"html.parser")
    temp_all_comment_text=[]
    all_comment=soup.find_all("div",class_="rj1gh0hx buofh1pr ni8dbmo4 stjgntxs hv4rvrfc")
    for comment in all_comment:
        record=[]
        time_label=comment.find("a",class_="oajrlxb2 g5ia77u1 qu0x051f esr5mh6w e9989ue4 r7d6kgcz rq0escxv nhd2j8a9 nc684nl6 p7hjln8o kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x jb3vyjys rz4wbd8a qt6c0cv9 a8nywdso i1ao9s8h esuyzwwr f1sip0of lzcic4wl m9osqain gpro0wi8 knj5qynh")
        time_text=time_label.find("span",class_="tojvnm2t a6sixzi8 abs2jz4q a8s20v7p t1p8iaqh k5wvi7nf q3lfd5jv pk4s997a bipmatt0 cebpdrjk qowsmv63 owwhemhu dp1hu0rb dhp61c6y iyyx5f41").get_text()
        record.append(time_text)
        record.append(comment.find("span",class_="d2edcug0 hpfvmrgz qv66sw1b c1et5uql lr9zc1uh a8c37x1j keod5gw0 nxhoafnm aigsh9s9 d9wwppkn fe6kdd0r mau55g9w c8b282yb mdeji52x e9vueds3 j5wam9gi lrazzd5p oo9gr5id").get_text())
        try:
            content=comment.find("div",class_="kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x c1et5uql").get_text()
        except AttributeError:
            record.append("picture")
        else:
            record.append(content)
            
        temp_all_comment_text.append(record)
        
    now=date.today()
    df=pd.DataFrame(temp_all_comment_text,columns=["Comment_Time","Comment_By","Comment"])
    this_post_date=this_post_date.replace("/","_")
    dest="C:/Users/Jim/Desktop/facebook_CDC_0520_post_"+str(count)+".csv"
    df.to_csv(dest,encoding='utf_8_sig')

In [4]:
#程式內時間處理

from datetime import date
from datetime import datetime
from datetime import timedelta

def time_dealing(current_time,input_string):
    if(input_string.find("小時")>-1):
        
        delta=timedelta(hours=int(input_string[:input_string.find("小時")]))
        output_time=current_time-delta
        output_string=output_time.strftime("%m月%d日 %H:%M:%S")
        
    elif (input_string.find("天")>-1):
        delta=timedelta(days=int(input_string[:input_string.find("天")]))
        output_time=current_time-delta
        output_string=output_time.strftime("%m月%d日 %H:%M:%S")
    else:
        output_string=input_string
    
    return output_string

In [5]:
#字串處理成時間

from datetime import date
from datetime import datetime
from datetime import timedelta

def date_dealing(input_string):
    input_string="2021年"+input_string
    cut_end=input_string.find("日")
    temp=input_string[:cut_end+1]
    output_time=datetime.strptime(temp,("%Y年%m月%d日"))
    
    return output_time

In [20]:
#test area

import pandas as pd

count=0
df = pd.read_csv('post_link_20210518_20210520.csv')
del df['A']
driver=OpenChromeDriver()
loginfb(driver)

for i in range(6,16):
    link=df.iloc[i,1]
    getPostComment(link,driver,df.iloc[i,0],count)
    count+=1
print("done")

done


In [28]:
df = pd.read_csv('post_link_20210518_20210520.csv')
del df['A']
for i in range(6,16):
    print(df.iloc[i,0],end=" ")
    print(df.iloc[i,1])

2021/5/20 https://www.facebook.com/TWCDC/posts/10159091036533407?__cft__[0]=AZXhMX8sQ36RD1MwaukF_e2Vg798Lu1eufCRqNiRzKqul_kGkBYwcMQhj9l-HB97o24hhCoyCzIlV4bfVwKLQIJJ9aw17FMcKEGd7KDmzFDIKzWX-KlmmgsOeAlD5alotwTrSgAVdzTClmHfYKY7yPMl&__tn__=%2CO%2CP-R
2021/5/20 https://www.facebook.com/TWCDC/posts/10159091120318407?__cft__[0]=AZXqhMevvnm_De0bjHq2MWH1G0YHprE64n0Pprhs_x0yWifZgbzKkXQ_hbfJQN_X_1Srx49kKOwX2mrBlLNZTXirOoAFbtdxwak_A4HcXb355jdEeEVGlpBdZGl9YPfngBfeiVKBFBYgmuXyYJykDyjPwEVIjHwarhU-fg275Kpgew&__tn__=%2CO%2CP-R
2021/5/20 https://www.facebook.com/TWCDC/posts/10159091076218407?__cft__[0]=AZU6eNGTmxQ8K6kxk5sraGgqgMo6e-iGG1mj-qax4DVSO8kQyF2nvo3GUnd0FJOM-dfgg6Qobc3OAXjNhgaEesVHqsBN0GbzX1VBPzAjIrR2e9DY4neKygVR9TKhjTDsfDvnjg-FZtbe2B5fsle0ZNLLFM5_TwDuIRqSC8a4hXSoVA&__tn__=%2CO%2CP-R
2021/5/20 https://www.facebook.com/TWCDC/posts/10159090777383407?__cft__[0]=AZX0ZAYSV_UkRLwNPVXsyl2rd-O6jQqqdVPbqMupG429SWXB9C8XGTpY3JVBlKWb2R_gfjt4GZWU2sP1qHE7dC2rDb9CqOXG2AJ7O2L5ZgpjJKpmSSncY0MSo1Ka5f0oDvSh_VtCdEv

unfunished 4,11,13,16,18,25,58,62,64,66,75,76 last:76